<a href="https://colab.research.google.com/github/danielmomodu/steama-backend-connect/blob/main/pp_geocoding_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import math
warnings.filterwarnings("ignore")

poles = pd.read_excel("POLE LAYOUT GPS (INI LGA).xlsx", skiprows=4).drop(columns=['Unnamed: 0','Unnamed: 1'])
poles.columns = ['Lat', 'Long', 'Status']
poles.head()
poles['Status'].unique()

array([nan, 'EXISTING'], dtype=object)

In [2]:
houses = pd.read_excel("Prado Power Customer enumeration    (Akwaibom State, Nigeria.).xlsx")
houses = houses[["9. Latitude ", "10. Longitude "]]
houses.columns=['Lat','Long']
houses.head()

,Lat,Long
0,5°21'38'',7°49'44''
1,5°21'13'',7°49'44''
2,5°21'12'',7°49'43''
3,5°21'13'',7°49'44''
4,5°21'13'',7°49'43''


In [3]:
def main(df,regex):
  def func(col):
    def f(x):
      col_str = x[col]
      col_list = re.split(regex,col_str)
      col_list = [int(i) for i in col_list[:3]]
      res = col_list[0] + col_list[1]/60 + col_list[2]/3600
      return res
    # df[[col+'_D',col+'_M',col+'_S']] = df.apply(f,axis=1,result_type="expand")
    df[[col+'_DD']] = df.apply(f,axis=1,result_type="expand")
  func('Lat')
  func('Long')
  # df["Lat_dir"] = "N"
  # df["Long_dir"] = "E"
  return df.head()

In [4]:
main(houses,"'|°").to_excel('houses_loc.xlsx')

,Lat,Long,Lat_DD,Long_DD
0,5°21'38'',7°49'44'',5.360556,7.828889
1,5°21'13'',7°49'44'',5.353611,7.828889
2,5°21'12'',7°49'43'',5.353333,7.828611
3,5°21'13'',7°49'44'',5.353611,7.828889
4,5°21'13'',7°49'43'',5.353611,7.828611


In [5]:
main(poles,"° |’ |”").to_excel('poles_loc.xlsx')

,Lat,Long,Status,Lat_DD,Long_DD
0,5° 21’ 2”N,7° 49’ 53”E,NaN,5.350556,7.831389
1,5° 21’ 3”N,7° 49’ 51”E,NaN,5.350833,7.830833
2,5° 21’ 4”N,7° 49’ 49”E,NaN,5.351111,7.830278
3,5° 21’ 5”N,7° 49’ 47”E,NaN,5.351389,7.829722
4,5° 21’ 6”N,7° 49’ 46”E,NaN,5.351667,7.829444


In [6]:
def dist(pos1,pos2):
  R = 6373.0
  (lat1,lon1)=pos1
  (lat2,lon2)= pos2
  dlon = abs(lon2 - lon1)
  dlat = abs(lat2 - lat1)
  a = math.sin(dlat / 2)*2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)*2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
  distance = R * c
  return distance

In [7]:
minimum=[]
for i in range(len(houses)):
  house = houses.iloc[i][['Lat_DD','Long_DD']].values
  dists=[]
  for j in range(len(poles)):
    pole = poles.iloc[j][['Lat_DD','Long_DD']].values
    dists.append(dist(house,pole))
  m=np.argmin(dists)
  minimum.append([m,dists[m]])
houses[['closer_pole_index','closer_distance']]=minimum

In [9]:
houses.to_excel('houses.xlsx')

In [10]:
poles.iloc[12]

Lat        5° 21’ 12”N
Long       7° 49’ 45”E
Status             NaN
Lat_DD         5.35333
Long_DD        7.82917
Name: 12, dtype: object

In [11]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored